In [1]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

print(es_client.info()['version']['build_hash'])

42f05b9372a9a4a470db3b52817899b99a76ee73


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
# defining the structure of the index in elasticsearch
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_faqs"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_faqs'})

In [5]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name,
                    document=doc)

100%|██████████| 948/948 [00:22<00:00, 41.24it/s]


In [6]:
query = "How do I execute a command in a running docker container?"

In [7]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [8]:
response = es_client.search(index=index_name, body=search_query)

In [9]:
res_score = []

for score in response['hits']['hits']:
    res_score.append(score['_score'])

In [10]:
res_score

[83.967926, 75.49801, 72.04509, 51.035336, 49.90406]

In [11]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question are three times more important
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": 'machine-learning-zoomcamp'
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

In [12]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [13]:
result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [14]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [15]:
context = ""

for doc in result_docs:
    context += f"Q: {doc['question']}\nA: {doc['text']}\n\n".strip()

In [16]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar AdvaniQ: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do i

In [17]:
prompt = prompt_template.format(question="How do I execute a command in a running docker container?",
                                 context=context).strip()

In [18]:
len(prompt)

1458

In [19]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.6 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
from tiktoken import encoding_for_model

In [22]:
encoding = encoding_for_model("gpt-4o")

In [25]:
len(encoding.encode(prompt))

321